# Usage of `src.models.fuel` module.

This notebook outlies the basic usage of the `src.models.fuel` module. Used to estimate the fuel usage of a given flight path
 
**Requirements**
 - A csv the flight's state vectors

**Helpful Links**


## Basic Setup

In [6]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

import sqlite3
import os
import pandas as pd

# Fuel Module
from src.models import fuel

# File containing some flight database
flights_database = os.path.normpath('../data/flight/KDEN_KSEA_2023-01-01_2023-01-31.sqlite')

# Creates the connection to the database
flights_connection = sqlite3.connect(flights_database)

# Reads the first flight_id and corresponding icao24
flight_id, icao24 = pd.read_sql_query("SELECT flight_id, icao24 FROM flights LIMIT 1;", flights_connection).values[0,:]

# Reads that flight's state vectors
state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """, flights_connection)
# Closes connection
flights_connection.close()

# Computes estimated mass, used fuel, and fuel flow at each point in time
state_vectors = fuel.compute_fuel_data(state_vectors, icao24)

state_vectors

,vector_id,flight_id,time,time_normalized,lat,lon,geoaltitude,baroaltitude,heading,velocity,mass,fuelflow,used_fuel
0,343194,ada167_1672524657_1672534041_KDEN_KSEA,1672524657,0,39.840848,-104.734741,1790.700000,1889.760000,270.000000,104.946576,79200.000000,0.898499,0.000000
1,343195,ada167_1672524657_1672534041_KDEN_KSEA,1672524658,1,39.840894,-104.735991,1806.751208,1905.492084,270.793074,104.910667,79199.101501,1.245082,0.898499
2,343196,ada167_1672524657_1672534041_KDEN_KSEA,1672524659,2,39.840940,-104.737241,1822.802417,1921.224169,271.586147,104.874757,79197.856419,1.245275,2.143581
3,343197,ada167_1672524657_1672534041_KDEN_KSEA,1672524660,3,39.840986,-104.738491,1838.853625,1936.956253,272.379221,104.838848,79196.611144,1.245468,3.388856
4,343198,ada167_1672524657_1672534041_KDEN_KSEA,1672524661,4,39.841032,-104.739741,1854.904834,1952.688338,273.172294,104.802939,79195.365676,1.245662,4.634324
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9380,352574,ada167_1672524657_1672534041_KDEN_KSEA,1672534037,9380,47.460375,-122.317874,116.414501,129.686163,180.388640,75.844079,74488.279820,1.308266,4711.720180
9381,352575,ada167_1672524657_1672534041_KDEN_KSEA,1672534038,9381,47.459666,-122.317881,113.980876,127.744623,180.388920,75.789314,74486.971554,1.310547,4713.028446
9382,352576,ada167_1672524657_1672534041_KDEN_KSEA,1672534039,9382,47.458957,-122.317889,111.547251,125.803082,180.389200,75.734548,74485.661007,1.312835,4714.338993
9383,352577,ada167_1672524657_1672534041_KDEN_KSEA,1672534040,9383,47.458248,-122.317896,109.113625,123.861541,180.389481,75.679783,74484.348172,1.315130,4715.651828


## Computing fuel quantities for all routes and flights

In [6]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

import sqlite3
import os
import pandas as pd
from IPython.display import clear_output

# Fuel Module
from src.models import fuel

files = ['../data/flight/' + f for f in os.listdir('../data/flight/') if f.endswith('.sqlite')]

new_columns = ['mass', 'fuelflow', 'used_fuel']

i = 0
for file in files:
    i += 1
    clear_output(wait=True)
    print(f'{i}/{len(files)}', file)
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    
    cursor.execute("DROP TABLE IF EXISTS state_vector_fuel;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_fuel (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query) 
    
    flight_ids, icao24s = pd.read_sql_query("SELECT flight_id, icao24 FROM flights;", conn).values.T
    
    j = 0
    for flight_id, icao24 in zip(flight_ids, icao24s):
        j += 1
        # Reads that flight's state vectors
        state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """, conn)
        state_vectors = fuel.compute_fuel_data(state_vectors, icao24)
        print(flight_id, state_vectors.iloc[-1]['used_fuel'], f'{j}/{len(flight_ids)}')
        for index, row in state_vectors.iterrows():
            # Preparing the data to be inserted
            insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

            # Creating query to insert new values
            insert_query = f'''
                INSERT INTO state_vector_fuel (vector_id, {', '.join(new_columns)})
                VALUES ({', '.join('?' * len(insert_data))})
                ON CONFLICT(vector_id) DO UPDATE SET
                {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
            '''
            cursor.execute(insert_query, insert_data)
        # Commiting changes to the database
        conn.commit()
    conn.close()

12/12 ../data/flight/KLAX_KJFK_2023-07-01_2023-07-31.sqlite
add734_1688153197_1688171334_KLAX_KJFK 10172.678335978737 1/967
a04641_1688153812_1688171718_KLAX_KJFK 10364.278598414792 2/967
ad1560_1688155909_1688173336_KLAX_KJFK 9905.32099829844 3/967
a14c29_1688162358_1688180489_KLAX_KJFK 17731.86476383706 4/967
a0563b_1688164596_1688181627_KLAX_KJFK 16815.749193085852 5/967
a01c37_1688165002_1688182528_KLAX_KJFK 9794.268689233752 6/967
ab398a_1688168979_1688186339_KLAX_KJFK 24839.45564869966 7/967
ab8fd0_1688188531_1688205473_KLAX_KJFK nan 8/967
a12861_1688190931_1688207860_KLAX_KJFK 16851.23987058844 9/967
acca19_1688192658_1688210969_KLAX_KJFK 10324.192585365963 10/967
a014c9_1688194620_1688212902_KLAX_KJFK 9955.589622161104 11/967
a12c18_1688196188_1688213896_KLAX_KJFK 17335.55039315473 12/967
ad1cde_1688199374_1688219544_KLAX_KJFK 11025.301272898534 13/967
adc9b6_1688199848_1688218329_KLAX_KJFK 10352.2059858664 14/967
add734_1688215691_1688234159_KLAX_KJFK 10422.626664663883 15/967

In [4]:
state_vectors

,vector_id,flight_id,time,time_normalized,lat,lon,geoaltitude,baroaltitude,heading,velocity,tmpf,air_pressure,air_density,clouds,sknt,mass,fuelflow,used_fuel
0,343194,ada167_1672524657_1672534041_KDEN_KSEA,1672524657,0,39.840848,-104.734741,1790.700000,1889.760000,270.000000,104.946576,39.576214,818.497805,1.028123,0.0,5.824382,79200.000000,0.898499,0.000000
1,343195,ada167_1672524657_1672534041_KDEN_KSEA,1672524658,1,39.840894,-104.735991,1806.751208,1905.492084,270.793074,104.910667,39.459827,817.551134,1.027160,0.0,6.020842,79199.101501,1.245082,0.898499
2,343196,ada167_1672524657_1672534041_KDEN_KSEA,1672524659,2,39.840940,-104.737241,1822.802417,1921.224169,271.586147,104.874757,39.343441,816.604463,1.026197,0.0,6.217303,79197.856419,1.245275,2.143581
3,343197,ada167_1672524657_1672534041_KDEN_KSEA,1672524660,3,39.840986,-104.738491,1838.853625,1936.956253,272.379221,104.838848,39.227054,815.657791,1.025234,0.0,6.413763,79196.611144,1.245468,3.388856
4,343198,ada167_1672524657_1672534041_KDEN_KSEA,1672524661,4,39.841032,-104.739741,1854.904834,1952.688338,273.172294,104.802939,39.110668,814.711120,1.024272,0.0,6.610223,79195.365676,1.245662,4.634324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9380,352574,ada167_1672524657_1672534041_KDEN_KSEA,1672534037,9380,47.460375,-122.317874,116.414501,129.686163,180.388640,75.844079,NaN,NaN,NaN,NaN,NaN,74488.279820,1.308266,4711.720180
9381,352575,ada167_1672524657_1672534041_KDEN_KSEA,1672534038,9381,47.459666,-122.317881,113.980876,127.744623,180.388920,75.789314,NaN,NaN,NaN,NaN,NaN,74486.971554,1.310547,4713.028446
9382,352576,ada167_1672524657_1672534041_KDEN_KSEA,1672534039,9382,47.458957,-122.317889,111.547251,125.803082,180.389200,75.734548,NaN,NaN,NaN,NaN,NaN,74485.661007,1.312835,4714.338993
9383,352577,ada167_1672524657_1672534041_KDEN_KSEA,1672534040,9383,47.458248,-122.317896,109.113625,123.861541,180.389481,75.679783,NaN,NaN,NaN,NaN,NaN,74484.348172,1.315130,4715.651828
